In [1]:
import ReadRawMHD as rR
import mapping as mapp
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import time
import os
t1 = time.time()

In [2]:
# # # # # Input # # # # #
# Input imaging data
model_code = '01_L50_S50_D35'  # FEA model name
sample_code = '03_Pilot3'  # Experimental sample. contains number and sample name, e.g. 00_Pilot3.
ExpScrew = 'P'  # T or P, site from experiment. Two input files (SimScrew Ti and PEEK) will be generated for each site
file_bone = 'XCT_Icotec_S130672_L5_intact_planned.mhd'  # gray value bone ct scan, gauss filter applied
file_inst = 'ICOTEC_S130672_L5_implants_XCTres.mhd'  # segmented screw scan

In [3]:
# Specify file locations
#---sample_no, sample = sample_code.split('_')  # sample name and number
path_project = '/home/biomech/Documents/01_Icotec/'  # General project folder
path_ct = path_project + '01_Experiments/02_Scans/' + sample_code + '/04_Registered/'  # Folder of CT data
path_fea = path_project + '02_FEA/98_Pilots/' + sample_code + '/' + model_code + '/'  # Folder of FEA files
# Include general path for mesh/template later
info = sample_code + ExpScrew + '_info.txt'  # .txt file containing info about landmarks. Specific structure

In [4]:
# # # # # Input # # # # #
# Input FEA data
Input = {}
Input['FEA_loc'] = path_fea  # path to FEA files
Input['Model_Code'] = model_code  # model code (FEA model name)
Input['Screw'] = ExpScrew  # which site of scan will be processed
Input['Resolution'] = 0.0607  # scan resolution, should be 0.0607

#Input['STL'] = path_fea + model_code + '_mesh.stl'  # stl mask of bone-ROI generated with abaqus-export
#Input['dummyMesh'] = path_fea + model_code + '_mesh.inp'  # path where dummy mesh will be saved
#Input['InputFile'] = path_fea + model_code + ExpScrew  # name given to input file
#Input['BoneElsets'] = path_fea + model_code + 'elsets' + ExpScrew + '.inp'  # elsets file temporarily saved here
# Input['BoneMaterials'] = path_fea + model_code +  'materials' + ExpScrew + '.inp'  # materials file temporarily saved here
#Input['Template'] = path_fea + model_code + '_model.inp'  # orig template file must exist here
#Input['Mask'] = path_fea + model_code + ExpScrew + '_mask.mhd'  # location where mask will be saved

Input['F_dir'] = '-'  # force direction (negative corresponds to experiment). Input '-' or '+'
Input['F_max'] = 150  # peak load
Input['F_min'] = 10  # valley load, not included yet
Input['Cycles'] = 2  # how many cycles to simulate, not included yet
Input['Friction'] = 0.2  # friction between screw and bone, not included yet
Input['ElType'] = 'Quadratic'  # element type quadratic or linear C3D8 = linear, not included yet

# parametrise screw length? --> different mesh sizes
# parametrise ROI?
# parametrise step size?

# Write output images? segmented image and mask, for visual check
write_output = 0

In [5]:
# Write mesh input file
mapp.write_mesh(Input)  # Original input file, path for mesh.inp
#--> template, ???

Finished extracting mesh file.


In [6]:
# Load data
imD = rR.load_itk(path_ct + file_inst)  # screw image
with open(path_ct + info, 'r') as f:
    content = f.read()
ori = content.split('Origin: ')[1].split('\n')[0]
ori = np.array([int(ori.split(' ')[0]), int(ori.split(' ')[1]), int(ori.split(' ')[2])])
p1P = content.split('positive x axis: ')[1].split('\n')[0]
p1P = np.array([int(p1P.split(' ')[0]), int(p1P.split(' ')[1]), int(p1P.split(' ')[2])])
p1N = content.split('negative x axis: ')[1].split('\n')[0]
p1N = np.array([int(p1N.split(' ')[0]), int(p1N.split(' ')[1]), int(p1N.split(' ')[2])])

In [7]:
# # # # # Input # # # # #
# Define screw vector
v3 = []
if ExpScrew == 'T':
    lineT = rR.axis3D(imD[0], 670, 1100, 'x')  # if starting from screw tip towards head, add (-) to v3
    v3 = -lineT.vector  # z-axis = screw axis, found by function. Specify +/-!!
elif ExpScrew == 'P':
    lineT = rR.axis3D(imD[0], 0, 600, 'x')  # if starting from screw tip towards head, add (-) to v3
    v3 = lineT.vector  # z-axis = screw axis, found by function. Specify +/-!!
del imD

In [8]:
# Compute matrix
v2 = np.cross(v3, p1P-p1N)/np.linalg.norm(np.cross(v3, p1P-p1N))  # y-axis
v1 = np.cross(v2, v3)  # x-axis
M = np.vstack((np.append(v1, ori[0]), np.append(v2, ori[1]), np.append(v3, ori[2]), np.array([0, 0, 0, 1])))
print(M)

[[-8.69530269e-01  7.30584938e-03 -4.93825613e-01  1.04000000e+02]
 [-1.12854213e-02  9.99335580e-01  3.46559802e-02  4.63000000e+02]
 [ 4.93750696e-01  3.57074539e-02 -8.68870087e-01  7.41000000e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
# Check coordinate system
# z-axis (blue) should point from screw head (origin) towards screw tip (dot)
# y-axis (green) should be perpendicular to screw-rotAxis plane and point upwards (against loading direction)
# x-axis (red): right-handed coordinate system
%matplotlib notebook
plt.figure()
ax = plt.axes(projection='3d')
fact = 100
for i in range(-0, 5):
    ax.scatter3D(ori[0]+i*fact*v1[0], ori[1]+i*fact*v1[1], ori[2]+i*fact*v1[2], c='r', alpha=1)
    ax.scatter3D(ori[0]+i*fact*v2[0], ori[1]+i*fact*v2[1], ori[2]+i*fact*v2[2], c='g', alpha=1)
    ax.scatter3D(ori[0]+i*fact*v3[0], ori[1]+i*fact*v3[1], ori[2]+i*fact*v3[2], c='b', alpha=1)
ax.scatter3D(p1P[0], p1P[1], p1P[2], c='b')
ax.scatter3D([0, 1500], [0, 1500], [0, 1500], alpha=0)
ax.scatter3D(686, 480, 177, c='k', alpha=1)  # Ti tip
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.show()

In [10]:
bone = {}

In [11]:
bone = mapp.readInpBoneDummy(bone, Input)  # Read bone mesh from abaqus. Read elements, nodes

In [12]:
bone = mapp.boneMeshMask(bone, Input)  # Create mask from abaqus bone mesh

BoneMeshMask


In [13]:
bone = mapp.load_BVTVdata(bone, path_ct + file_bone)

In [14]:
# Read mask
imMask = sitk.ReadImage(Input['FEA_loc'] + Input['Model_Code'] + Input['Screw'] + '_mask.mhd')
imMask_np = np.transpose(sitk.GetArrayFromImage(imMask), [2, 1, 0])
bone["GreyImage"].SetOrigin([0, 0, 0])
# Define rotation and translation
[theta1, theta2, theta3] = rR.rotation_angles_from_matrix(M[:3, :3], 'zyx')
theta2 = -theta2 + np.pi  # +pi because arc-tan has no unique solution. WHY NEGATIVE, because mask[:, ::, :] reshaped?
center = np.array([imMask_np.shape[0]/2, imMask_np.shape[1]/2, 0]) * imMask.GetSpacing()
trans = M[:3, 3] * imMask.GetSpacing() - center

In [15]:
# Write transformation file
f = open(path_fea + 'transformation_' + model_code + ExpScrew + '.tfm', "w")
f.write(
    "#Insight Transform File V1.0\n"
    "#Transform 0\n"
    "Transform: CompositeTransform_double_3_3\n"
    "#Transform 1\n"
    "Transform: Euler3DTransform_double_3_3\n"
    "Parameters:  " + f'{theta1}' + " " + f'{theta2}' + " " + f'{theta3}'
    + " " + f'{trans[0]}' + " " + f'{trans[1]}' + " " + f'{trans[2]}' + "\n"  # transformation
    "FixedParameters: " + f'{center[0]}' + " " + f'{center[1]}' + " " + f'{center[2]}' + " 0\n")  # CoR
f.close()

In [16]:
# Apply transformation to mask using the inverse transformation
bone['Transform'] = sitk.ReadTransform(path_fea + 'transformation_' + model_code + ExpScrew + '.tfm')
bone['Transform_inv'] = bone['Transform'].GetInverse()
imMask_trans = sitk.Resample(imMask, bone['GreyImage'], bone['Transform_inv'], sitk.sitkNearestNeighbor, 0.0, bone['GreyImage'].GetPixelID())
# Delete some files / variables to save memory
os.remove(path_fea + model_code + ExpScrew + '_mask.mhd')
os.remove(path_fea + model_code + ExpScrew + '_mask.raw')
if write_output:
    img_seg=sitk.GetImageFromArray(np.transpose(bone['BVTVscaled'], [2, 1, 0]))
    img_seg.SetOrigin(bone["GreyImage"].GetOrigin())
    img_seg.SetSpacing(bone["GreyImage"].GetSpacing())
    sitk.WriteImage(img_seg, path_fea+ + sample_code + '_seg.mhd')
    imMask_trans.SetOrigin(bone["GreyImage"].GetOrigin())
    imMask_trans.SetSpacing(bone["GreyImage"].GetSpacing())
    sitk.WriteImage(imMask_trans, path_fea + model_code + '_' + ExpScrew + '_maskTrans.mhd')
    del img_seg
del bone['GreyImage']

In [17]:
# BVTV segmentation / calibration HR-pQCT to uCT
bone['MASK_array_T'] = np.transpose(sitk.GetArrayFromImage(imMask_trans), [2, 1, 0])
# scaling factor/intercept from Schenk et al. 2022, has to be discussed w Ph

In [18]:
#rR.blockPrint()
mapp.HFE_mapping_trans(bone, Input)
#rR.enablePrint()

... start material mapping with copying boundary layers as ghost layers
FEelSize material mapping = [5.2224634590587735, 4.732317186923472, 5.219496425676788, 4.730901082823314, 5.354912916817603, 4.619966343349743, 5.355137588142736, 4.612949903500818, 5.880101318408981, 4.321324255147204, 5.871639968630036, 4.313209430546461, 4.739668201328267, 5.22150051292873, 4.61761375628934, 5.336304000655688, 4.3273652747668905, 5.862769538673613, 5.561408861982365, 5.858304147024175, 6.8168756791826155, 5.56428460377971, 5.559123905903529, 5.863057245533634, 5.845116815824281, 6.809674324630631, 6.80552767200423, 3.2650445671370605, 1.9392669828149551, 3.263164884140134, 1.9241590029305216, 4.150589460918798, 1.8542354692684992, 4.148297745150885, 1.8642714844899841, 1.9371831185991966, 3.27404045290901, 1.8604245003644755, 4.151572359047667, 5.938131081099195, 4.185758291465851, 5.9318862520942695, 4.18530839208358, 3.0789639714849066, 1.8595999347457284, 3.9963451984710683, 1.821519169168890

/home/biomech/Documents/01_Icotec/04_Code/venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/biomech/Documents/01_Icotec/04_Code/venv/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


 ... material mapping element 2679/2680... material mapping finished

 ... update ABAQUS file       :  /home/biomech/Documents/01_Icotec/02_FEA/98_Pilots/03_Pilot3/01_L50_S50_D35/01_L50_S50_D35P_elsets.inp and /home/biomech/Documents/01_Icotec/02_FEA/98_Pilots/03_Pilot3/01_L50_S50_D35/01_L50_S50_D35P_materials.inp


{'elems': {1: <MedtoolFunctions.med_classes.element at 0x7f1978451570>,
  2: <MedtoolFunctions.med_classes.element at 0x7f19784515a0>,
  3: <MedtoolFunctions.med_classes.element at 0x7f19784515d0>,
  4: <MedtoolFunctions.med_classes.element at 0x7f1978451600>,
  5: <MedtoolFunctions.med_classes.element at 0x7f1978451630>,
  6: <MedtoolFunctions.med_classes.element at 0x7f1978451660>,
  7: <MedtoolFunctions.med_classes.element at 0x7f1978451690>,
  8: <MedtoolFunctions.med_classes.element at 0x7f19784516c0>,
  9: <MedtoolFunctions.med_classes.element at 0x7f19784516f0>,
  10: <MedtoolFunctions.med_classes.element at 0x7f1978451720>,
  11: <MedtoolFunctions.med_classes.element at 0x7f1978451750>,
  12: <MedtoolFunctions.med_classes.element at 0x7f1978451780>,
  13: <MedtoolFunctions.med_classes.element at 0x7f19784517b0>,
  14: <MedtoolFunctions.med_classes.element at 0x7f19784517e0>,
  15: <MedtoolFunctions.med_classes.element at 0x7f1978451810>,
  16: <MedtoolFunctions.med_classes.elem

In [19]:
sliceNo = 460
plt.imshow(bone['BVTVscaled'][:, sliceNo, :] + bone['MASK_array_T'][:, sliceNo, :])

plt.show()

In [20]:
# Write final input file
mapp.HFE_inp_creator(Input)
os.remove(Input['FEA_loc'] + Input['Model_Code'] + Input['Screw'] + '_elsets.inp')
os.remove(Input['FEA_loc'] + Input['Model_Code'] + Input['Screw'] + '_materials.inp')

Section set to Ti.
found set-rp
1
step2
found set-rp
2
Section set to PEEK.
found set-rp
1
step2
found set-rp
2
End HFE_inp_creator


In [21]:
print('Execution time: ' + str(int((time.time()-t1)/60)) + ' min '+ str(round(np.mod(time.time()-t1, 60), 1)) + ' sec.')

Execution time: 2 min 42.4 sec.
